In [2]:
import streamlit as st
import pandas as pd
import plotly.express as px
from datetime import datetime
import pytz

st.set_page_config(page_title="Global App Installs", layout="wide")
st.title("🌍 Global App Installs – Choropleth Map")

# ---------------- LOAD DATA ----------------
df = pd.read_csv("googleplaystore.csv")

# Normalize column names
df.columns = df.columns.str.strip()

# ---------------- CLEAN INSTALLS ----------------
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace(r"[+,]", "", regex=True)
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

df = df.dropna(subset=["Installs", "Category"])

# ---------------- COUNTRY FIX ----------------
if "Country" not in df.columns:
    df["Country"] = "India"

# ---------------- FILTER CATEGORY ----------------
df = df[~df["Category"].str.startswith(tuple("ACGS"))]
df = df[df["Installs"] > 1_000_000]

# ---------------- TOP 5 CATEGORIES ----------------
top_categories = (
    df.groupby("Category")["Installs"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
    .index
)

df = df[df["Category"].isin(top_categories)]

# ---------------- TIME CONTROL (TEST MODE) ----------------
current_hour = 18  # FORCE GRAPH TO SHOW

# ---------------- CHOROPLETH ----------------
map_df = df.groupby(["Country", "Category"], as_index=False)["Installs"].sum()

fig = px.choropleth(
    map_df,
    locations="Country",
    locationmode="country names",
    color="Installs",
    hover_name="Category",
    color_continuous_scale="Viridis",
    title="Global Installs by Top 5 App Categories"
)

st.plotly_chart(fig, use_container_width=True)

2026-02-13 22:32:38.343 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 22:32:38.346 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 22:32:38.347 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-02-13 22:32:38.349 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
C:\Users\GOD\AppData\Local\Temp\ipykernel_11520\3344885751.py:51: DeprecationWarning: The library used by the *country names* `locationmode` option is changing in an upcoming version. Country names in existing plots may not work in the new version. To ensure consistent behavior, consider setting `locationmode` to *ISO-3*.
  fig = px.choropleth(
2026-02-13 22:32:47.215 Please replace `use_container_width` with `width`.

`use_container_width` will be removed after 2025-12-31.

For `use_contain

DeltaGenerator()